# **Distribución % de créditos por vivienda valor**

In [1]:
# Manejo de datos
import os # Directorios
import pandas as pd # Manipulación df
# Gráficas
import plotly.graph_objects as go #Para obtener librería usar: pip install plotly
from plotly.subplots import make_subplots
import plotly.io as pio # Exportar gráfica

# Obtener el directorio actual de trabajo
directorio_actual = os.getcwd()

# Directorio donde se encuentran los archivos JSON (ruta relativa)
directorio_json = os.path.join(directorio_actual, '../../db/datos_json')

# Obtener la lista de archivos JSON en el directorio
archivos_json = os.listdir(directorio_json)

# Cargar los archivos JSON y crear DataFrames
for archivo in archivos_json:
    nombre_tabla = archivo.replace('datos_', '').replace('.json', '')
    ruta_json = os.path.join(directorio_json, archivo)
    globals()[f"df_{nombre_tabla}"] = pd.read_json(ruta_json)

# Obtener todos los nombres de las variables globales
nombres_variables_globales = list(globals().keys())

# Filtrar los nombres que comienzan con "df_", contienen "alfa_q" y "pachuca"
nombres_df_filtrados = [
    nombre for nombre in nombres_variables_globales 
    # Caso de cuando no son las alfa q
    if nombre.startswith("df_") and "financiamientos" in nombre and "tulancingo" in nombre
]

# Imprimir la lista de DataFrames filtrados
print("Lista de DataFrames filtrados:")
nombres_df_filtrados

Lista de DataFrames filtrados:


['df_financiamientos_2019_tulancingo',
 'df_financiamientos_2020_tulancingo',
 'df_financiamientos_2021_tulancingo',
 'df_financiamientos_2022_tulancingo',
 'df_financiamientos_2023_tulancingo',
 'df_financiamientos_2024_tulancingo']

## 2022

In [10]:
finance = df_financiamientos_2022_tulancingo[['id','vivienda_valor']]
# Convertir la columna 'organismo' a cadena de texto (str)
finance['vivienda_valor'] = finance['vivienda_valor'].astype(str)
# Crear un diccionario con las correspondencias entre número y categoría
categorias = {
    '1': 'Económica',
    '2': 'Popular',
    '3': 'Tradicional',
    '4': 'Media',
    '5': 'Residencial',
    '6': 'Residencial Plus'
}
# Reemplazar los valores de 'organismo' con el texto correspondiente utilizando replace
finance['vivienda_valor'] = finance['vivienda_valor'].replace(categorias)

# Reemplazar cualquier otro número que no esté en el diccionario con '-'
finance['vivienda_valor'] = finance['vivienda_valor'].replace(to_replace=r'^\d+$', value='-', regex=True)

# Obtener el conteo de registros por cada categoría en la columna 'organismo'
conteo_organismos = finance['vivienda_valor'].value_counts().reset_index()
conteo_organismos.columns = ['vivienda_valor', 'conteo']

# Asegurarse de que todas las categorías estén presentes en el conteo, incluso si tienen un conteo de 0
for categoria in categorias.values():
    if categoria not in conteo_organismos['vivienda_valor'].unique():
        #conteo_organismos = conteo_organismos.append({'vivienda_valor': categoria, 'conteo': 0}, ignore_index=True)
        nuevo_registro = pd.DataFrame({'vivienda_valor': [categoria], 'conteo': [0]})
        conteo_organismos = pd.concat([conteo_organismos, nuevo_registro], ignore_index=True)

# Ordenar el DataFrame por las categorías en orden ascendente
conteo_organismos = conteo_organismos.sort_values(by='vivienda_valor').reset_index(drop=True)
conteo_organismos

C:\Users\yoe11\AppData\Local\Temp\ipykernel_3572\1669501074.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\yoe11\AppData\Local\Temp\ipykernel_3572\1669501074.py:14: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\yoe11\AppData\Local\Temp\ipykernel_3572\1669501074.py:17: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas

,vivienda_valor,conteo
0,-,29
1,Económica,14
2,Media,101
3,Popular,38
4,Residencial,25
5,Residencial Plus,1
6,Tradicional,108


In [11]:
# Crear un DataFrame con el conteo de registros por cada organismo
creditos_vivienda = finance['vivienda_valor'].value_counts().reset_index()
creditos_vivienda.columns = ['vivienda_valor', 'conteo']

# Calcular el total de registros en el DataFrame original
total_registros = len(finance)

# Calcular el promedio que ocupa cada organismo con respecto al total de registros
creditos_vivienda['promedio'] = creditos_vivienda['conteo'] / total_registros
creditos_vivienda

,vivienda_valor,conteo,promedio
0,Tradicional,108,0.341772
1,Media,101,0.319620
2,Popular,38,0.120253
3,-,29,0.091772
4,Residencial,25,0.079114
5,Económica,14,0.044304
6,Residencial Plus,1,0.003165


In [12]:
creditos_vivienda = creditos_vivienda[creditos_vivienda['vivienda_valor'] != '-']

# Definir el diccionario de orden de organismos
orden_viv = {
    'Económica':1, 
    'Popular':2,
    'Tradicional':3,
    'Media':4,
    'Residencial':5,
    'Residencial Plus':6
}

creditos_vivienda['orden'] = creditos_vivienda['vivienda_valor'].map(orden_viv)
# Ordenar el DataFrame por el orden de los organismos
creditos_vivienda = creditos_vivienda.sort_values(by='orden')
creditos_vivienda = creditos_vivienda.drop(columns=['orden'])
creditos_vivienda

C:\Users\yoe11\AppData\Local\Temp\ipykernel_3572\531983862.py:13: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,vivienda_valor,conteo,promedio
5,Económica,14,0.044304
2,Popular,38,0.120253
0,Tradicional,108,0.341772
1,Media,101,0.319620
4,Residencial,25,0.079114
6,Residencial Plus,1,0.003165


In [13]:
labels = creditos_vivienda['vivienda_valor']
values = creditos_vivienda['promedio']
colores = ['#2962ff', '#9500ff', '#ff0059', '#ff8c00', '#b4e600', '#2EC2A2','#E50CB6']

fig = go.Figure()
fig.add_trace(go.Bar(
        x=labels, 
        y=values, 
        marker_color=colores,
        text=values,  # Texto que se mostrará en las barras (porcentaje)
        textposition='outside',  # Posición del texto (puede ser 'inside' o 'outside')
        texttemplate='%{text:.2f}%', 
    )
)    
fig.update_layout(
    #barmode='stack', 
    #title='Distribución de créditos por vivienda valor (2022)',
    #title_font_size = 22,
    yaxis=dict(
        #title='M', # Título del eje y
        gridcolor='#dddcda', # Color de las líneas que dividen los rangos del eje Y         
    ),  margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes (left, right, top, bottom)
    plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
    legend=dict(
        yanchor="top",
        y=0.99,
    )
)

# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets\graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html')
guardar_grafico_como_html(fig, 'g_bar_creditos_vivienda_2022', carpeta='assets\graficas')
fig.show()

## 2023

In [6]:
finance = df_financiamientos_2023_tulancingo[['id','vivienda_valor']]
# Convertir la columna 'organismo' a cadena de texto (str)
finance['vivienda_valor'] = finance['vivienda_valor'].astype(str)
# Crear un diccionario con las correspondencias entre número y categoría
categorias = {
    '1': 'Económica',
    '2': 'Popular',
    '3': 'Tradicional',
    '4': 'Media',
    '5': 'Residencial',
    '6': 'Residencial Plus'
}
# Reemplazar los valores de 'organismo' con el texto correspondiente utilizando replace
finance['vivienda_valor'] = finance['vivienda_valor'].replace(categorias)

# Reemplazar cualquier otro número que no esté en el diccionario con '-'
finance['vivienda_valor'] = finance['vivienda_valor'].replace(to_replace=r'^\d+$', value='-', regex=True)

# Obtener el conteo de registros por cada categoría en la columna 'organismo'
conteo_organismos = finance['vivienda_valor'].value_counts().reset_index()
conteo_organismos.columns = ['vivienda_valor', 'conteo']

# Asegurarse de que todas las categorías estén presentes en el conteo, incluso si tienen un conteo de 0
for categoria in categorias.values():
    if categoria not in conteo_organismos['vivienda_valor'].unique():
        #conteo_organismos = conteo_organismos.append({'vivienda_valor': categoria, 'conteo': 0}, ignore_index=True)
        nuevo_registro = pd.DataFrame({'vivienda_valor': [categoria], 'conteo': [0]})
        conteo_organismos = pd.concat([conteo_organismos, nuevo_registro], ignore_index=True)

# Ordenar el DataFrame por las categorías en orden ascendente
conteo_organismos = conteo_organismos.sort_values(by='vivienda_valor').reset_index(drop=True)
conteo_organismos

C:\Users\yoe11\AppData\Local\Temp\ipykernel_3572\155678818.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\yoe11\AppData\Local\Temp\ipykernel_3572\155678818.py:14: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\yoe11\AppData\Local\Temp\ipykernel_3572\155678818.py:17: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-do

,vivienda_valor,conteo
0,-,25
1,Económica,15
2,Media,101
3,Popular,49
4,Residencial,15
5,Residencial Plus,6
6,Tradicional,101


In [7]:
# Crear un DataFrame con el conteo de registros por cada organismo
creditos_vivienda = finance['vivienda_valor'].value_counts().reset_index()
creditos_vivienda.columns = ['vivienda_valor', 'conteo']

# Calcular el total de registros en el DataFrame original
total_registros = len(finance)

# Calcular el promedio que ocupa cada organismo con respecto al total de registros
creditos_vivienda['promedio'] = creditos_vivienda['conteo'] / total_registros
creditos_vivienda

,vivienda_valor,conteo,promedio
0,Media,101,0.323718
1,Tradicional,101,0.323718
2,Popular,49,0.157051
3,-,25,0.080128
4,Residencial,15,0.048077
5,Económica,15,0.048077
6,Residencial Plus,6,0.019231


In [8]:
creditos_vivienda = creditos_vivienda[creditos_vivienda['vivienda_valor'] != '-']

# Definir el diccionario de orden de organismos
orden_viv = {
    'Económica':1, 
    'Popular':2,
    'Tradicional':3,
    'Media':4,
    'Residencial':5,
    'Residencial Plus':6
}

creditos_vivienda['orden'] = creditos_vivienda['vivienda_valor'].map(orden_viv)
# Ordenar el DataFrame por el orden de los organismos
creditos_vivienda = creditos_vivienda.sort_values(by='orden')
creditos_vivienda = creditos_vivienda.drop(columns=['orden'])
creditos_vivienda

C:\Users\yoe11\AppData\Local\Temp\ipykernel_3572\531983862.py:13: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,vivienda_valor,conteo,promedio
5,Económica,15,0.048077
2,Popular,49,0.157051
1,Tradicional,101,0.323718
0,Media,101,0.323718
4,Residencial,15,0.048077
6,Residencial Plus,6,0.019231


In [9]:
labels = creditos_vivienda['vivienda_valor']
values = creditos_vivienda['promedio']
colores = ['#2962ff', '#9500ff', '#ff0059', '#ff8c00', '#b4e600', '#2EC2A2','#E50CB6']

fig = go.Figure()
fig.add_trace(go.Bar(
        x=labels, 
        y=values, 
        marker_color=colores,
        text=values,  # Texto que se mostrará en las barras (porcentaje)
        textposition='outside',  # Posición del texto (puede ser 'inside' o 'outside')
        texttemplate='%{text:.2f}%', 
    )
)    
fig.update_layout(
    #barmode='stack', 
    #title='Distribución de créditos por vivienda valor (2022)',
    #title_font_size = 22,
    yaxis=dict(
        #title='M', # Título del eje y
        gridcolor='#dddcda', # Color de las líneas que dividen los rangos del eje Y         
    ),  margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes (left, right, top, bottom)
    plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
    legend=dict(
        yanchor="top",
        y=0.99,
    )
)

# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets\graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html')
guardar_grafico_como_html(fig, 'g_bar_creditos_vivienda_2023', carpeta='assets\graficas')
fig.show()

## 2024

In [2]:
finance = df_financiamientos_2024_tulancingo[['id','vivienda_valor']]
# Convertir la columna 'organismo' a cadena de texto (str)
finance['vivienda_valor'] = finance['vivienda_valor'].astype(str)
# Crear un diccionario con las correspondencias entre número y categoría
categorias = {
    '1': 'Económica',
    '2': 'Popular',
    '3': 'Tradicional',
    '4': 'Media',
    '5': 'Residencial',
    '6': 'Residencial Plus'
}
# Reemplazar los valores de 'organismo' con el texto correspondiente utilizando replace
finance['vivienda_valor'] = finance['vivienda_valor'].replace(categorias)

# Reemplazar cualquier otro número que no esté en el diccionario con '-'
finance['vivienda_valor'] = finance['vivienda_valor'].replace(to_replace=r'^\d+$', value='-', regex=True)

# Obtener el conteo de registros por cada categoría en la columna 'organismo'
conteo_organismos = finance['vivienda_valor'].value_counts().reset_index()
conteo_organismos.columns = ['vivienda_valor', 'conteo']

# Asegurarse de que todas las categorías estén presentes en el conteo, incluso si tienen un conteo de 0
for categoria in categorias.values():
    if categoria not in conteo_organismos['vivienda_valor'].unique():
        #conteo_organismos = conteo_organismos.append({'vivienda_valor': categoria, 'conteo': 0}, ignore_index=True)
        nuevo_registro = pd.DataFrame({'vivienda_valor': [categoria], 'conteo': [0]})
        conteo_organismos = pd.concat([conteo_organismos, nuevo_registro], ignore_index=True)

# Ordenar el DataFrame por las categorías en orden ascendente
conteo_organismos = conteo_organismos.sort_values(by='vivienda_valor').reset_index(drop=True)
conteo_organismos

C:\Users\yoe11\AppData\Local\Temp\ipykernel_3572\954197302.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  finance['vivienda_valor'] = finance['vivienda_valor'].astype(str)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_3572\954197302.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  finance['vivienda_valor'] = finance['vivienda_valor'].replace(categorias)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_3572\954197302.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Da

,vivienda_valor,conteo
0,-,47
1,Económica,1
2,Media,53
3,Popular,30
4,Residencial,11
5,Residencial Plus,3
6,Tradicional,52


In [3]:
# Crear un DataFrame con el conteo de registros por cada organismo
creditos_vivienda = finance['vivienda_valor'].value_counts().reset_index()
creditos_vivienda.columns = ['vivienda_valor', 'conteo']

# Calcular el total de registros en el DataFrame original
total_registros = len(finance)

# Calcular el promedio que ocupa cada organismo con respecto al total de registros
creditos_vivienda['promedio'] = creditos_vivienda['conteo'] / total_registros
creditos_vivienda

,vivienda_valor,conteo,promedio
0,Media,53,0.269036
1,Tradicional,52,0.263959
2,-,47,0.238579
3,Popular,30,0.152284
4,Residencial,11,0.055838
5,Residencial Plus,3,0.015228
6,Económica,1,0.005076


In [4]:
creditos_vivienda = creditos_vivienda[creditos_vivienda['vivienda_valor'] != '-']

# Definir el diccionario de orden de organismos
orden_viv = {
    'Económica':1, 
    'Popular':2,
    'Tradicional':3,
    'Media':4,
    'Residencial':5,
    'Residencial Plus':6
}

creditos_vivienda['orden'] = creditos_vivienda['vivienda_valor'].map(orden_viv)
# Ordenar el DataFrame por el orden de los organismos
creditos_vivienda = creditos_vivienda.sort_values(by='orden')
creditos_vivienda = creditos_vivienda.drop(columns=['orden'])
creditos_vivienda

C:\Users\yoe11\AppData\Local\Temp\ipykernel_3572\531983862.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  creditos_vivienda['orden'] = creditos_vivienda['vivienda_valor'].map(orden_viv)


,vivienda_valor,conteo,promedio
6,Económica,1,0.005076
3,Popular,30,0.152284
1,Tradicional,52,0.263959
0,Media,53,0.269036
4,Residencial,11,0.055838
5,Residencial Plus,3,0.015228


In [5]:
labels = creditos_vivienda['vivienda_valor']
values = creditos_vivienda['promedio']
colores = ['#2962ff', '#9500ff', '#ff0059', '#ff8c00', '#b4e600', '#2EC2A2','#E50CB6']

fig = go.Figure()
fig.add_trace(go.Bar(
        x=labels, 
        y=values, 
        marker_color=colores,
        text=values,  # Texto que se mostrará en las barras (porcentaje)
        textposition='outside',  # Posición del texto (puede ser 'inside' o 'outside')
        texttemplate='%{text:.2f}%', 
    )
)    
fig.update_layout(
    #barmode='stack', 
    #title='Distribución de créditos por vivienda valor (2022)',
    #title_font_size = 22,
    yaxis=dict(
        #title='M', # Título del eje y
        gridcolor='#dddcda', # Color de las líneas que dividen los rangos del eje Y         
    ),  margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes (left, right, top, bottom)
    plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
    legend=dict(
        yanchor="top",
        y=0.99,
    )
)

# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets\graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html')
guardar_grafico_como_html(fig, 'g_bar_creditos_vivienda_2024', carpeta='assets\graficas')
fig.show()